In [ ]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

在Vertex AI上使用Cloud Storage中的数据训练PyTorch模型

在Colab中打开
在Colab企业版中打开
在Workbench中打开
在GitHub上查看

## 概述

本教程向您展示如何使用PyTorch和存储在云存储中的数据集创建自定义训练作业。

了解更多关于[在Vertex AI中集成PyTorch](https://cloud.google.com/vertex-ai/docs/start/pytorch)的信息。

目标

在本教程中，您将学习如何使用PyTorch和存储在云存储上的数据集创建一个训练作业。您将构建一个使用GCSFuse加载存储桶中数据的自定义训练脚本。该自定义训练脚本创建了一个简单的神经网络，并将模型工件保存到云存储桶中。

本教程使用以下Vertex AI服务：

- Vertex AI 训练

执行的步骤包括：

- 编写一个创建训练集和测试集并训练模型的自定义训练脚本。
- 使用Vertex AI SDK for Python 运行"CustomTrainingJob"。

### 数据集

本教程使用[MNIST手写样本](https://en.wikipedia.org/wiki/MNIST_database)，该数据集对手写数字进行分类。对于本教程，数据集的CSV版本已经上传到了一个云存储桶中，可供您使用。您可以在[Kaggle](https://www.kaggle.com/datasets/oddrationale/mnist-in-csv?select=mnist_train.csv)上找到该数据集。

### 费用

本教程使用 Google Cloud 的可计费组件：

* Vertex AI
* Cloud Storage

了解[Vertex AI 价格](https://cloud.google.com/vertex-ai/pricing)，以及[Cloud Storage 价格](https://cloud.google.com/storage/pricing)，并使用[定价计算器](https://cloud.google.com/products/calculator/)根据您的预计使用量生成成本估算。

## 要求

这个教程需要使用一个针对PyTorch优化的笔记本电脑。如果您在Vertex AI Workbench中运行此笔记本，请确保您的笔记本镜像符合以下要求：

+ PyTorch 1.13 笔记本
+ 1个NVIDIA T4 GPU

Colab 笔记本符合要求（在安装和验证之后）。您可能需要切换到启用GPU的运行时。

开始吧!

### 为Python安装Vertex AI SDK和其他必需的软件包

### 安装Vertex AI SDK for Python和其他必需的软件包

In [ ]:
! pip3 install --upgrade --quiet google-cloud-aiplatform \
                                google-cloud-storage \
                                torch==1.13

重新启动运行时（仅限Colab）

要使用新安装的软件包，您必须在Google Colab上重新启动运行时。

In [ ]:
import sys

if "google.colab" in sys.modules:

    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

<div class="alert alert-block alert-warning">
<b>⚠️ 内核将重新启动。在继续下一步之前，请等待它完成。⚠️</b>
</div>

### 验证您的笔记本环境（仅限Colab）

在Google Colab上验证您的环境。

In [ ]:
import sys

if "google.colab" in sys.modules:

    from google.colab import auth

    auth.authenticate_user()

### 设置Google Cloud项目信息

了解更多关于[设置项目和开发环境](https://cloud.google.com/vertex-ai/docs/start/cloud-environment)。

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}

创建一个云存储桶

创建一个存储桶来存储中间产物，如数据集。

In [ ]:
BUCKET_URI = f"gs://your-bucket-name-{PROJECT_ID}-unique"  # @param {type:"string"}
BUCKET_PREFIX = "pytorch-on-gcs"  # @param

如果您的存储桶尚不存在：运行下面的单元格来创建您的云存储存储桶。

In [ ]:
! gsutil mb -l {LOCATION} -p {PROJECT_ID} {BUCKET_URI}

初始化用于Python的Vertex AI SDK

要开始使用Vertex AI，您必须拥有一个现有的Google Cloud项目并[启用Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com)。

In [ ]:
from google.cloud import aiplatform

aiplatform.init(project=PROJECT_ID, location=LOCATION, staging_bucket=BUCKET_URI)

### 导入库

In [ ]:
import os
from datetime import datetime

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from google.cloud import aiplatform
from matplotlib import pyplot as plt
from torch.utils.data import DataLoader, Dataset

提供数据的URI

如前所述，本教程使用经典的MNIST手写数字数据集作为输入。该数据集已经存储在一个公共可用的云存储位置上供您使用。您可以直接在训练脚本中使用这些CSV文件。

**注意**：您可以使用[PyTorch数据集库](https://pytorch.org/vision/stable/generated/torchvision.datasets.MNIST.html#torchvision.datasets.MNIST)下载该数据集。为了学习目的，本教程在云存储上使用了该数据集的副本。

In [ ]:
TRAIN_URI = "gs://cloud-samples-data/vertex-ai/training/pytorch/mnist_train.csv"
TEST_URI = "gs://cloud-samples-data/vertex-ai/training/pytorch/mnist_test.csv"

print(TRAIN_URI)
print(TEST_URI)

# [可选] 检查来自云存储的数据集

在创建训练脚本之前，快速查看一下云存储中的CSV文件中包含的数据。您可以使用 PyTorch 中的 `Dataset` 和 `DataLoader` 类来实例化一个数据集，然后使用 [matplotlib](https://matplotlib.org/stable/index.html) 来绘制数据。

首先将CSV文件下载到您的本地开发环境中。

In [ ]:
! gsutil -m cp -r $TRAIN_URI .
! gsutil -m cp -r $TEST_URI .

接下来，您需要定义一个继承自基类`Dataset`的自定义图像数据集。

请注意，您的自定义`Dataset`类必须重写`__init__`，`__len__`和`__getitem__`方法。这些方法是由`DataLoader`类使用的，用于遍历数据集。

以下`CustomImageDataset`类具有几个显著特点。首先，图像的尺寸被硬编码为28像素高，28像素宽。这对应于MNIST数据集中图像的尺寸。

其次，该数据集使用[`pandas.Dataframe`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html)对象来读取CSV文件并访问其中的数据。

最后，`__getitem__`方法需要从CSV文件中的每一行提取图像和标签。这两个值作为元组提供给调用者。

图像本身需要从一维向量值（列表）转换为二维矩阵（列表的列表）。此外，CSV文件中以整数形式存储的灰度值需要转换为介于0.0和1.0之间的浮点值。要进行此转换，您将灰度值乘以1/255的十进制等价值。

In [ ]:
class CustomImageDataset(Dataset):
    width = 28  # hard-coded width & height of image matrix
    height = 28

    def __init__(self, data_file, transform=None, target_transform=None):
        self.dataset = pd.read_csv(data_file)
        self.transform = (
            transform  # We would use ToTensor() if we were taking in raw images
        )
        self.target_transform = target_transform

    def __len__(self):
        return self.dataset.shape[0]

    def __getitem__(self, idx):
        label = self.dataset.at[idx, "label"]
        image = self.dataset.iloc[idx, 1:]

        # Create a matrix from the pandas.Series
        image = image.to_numpy() * 0.00392156862745098  # 1 / 255
        image = image.reshape(self.width, self.height)
        image = image.astype(float)
        image = torch.Tensor(image)

        if self.target_transform:
            label = self.target_transform(label)
        if self.transform:
            image = self.transform(image)
        return image, label

In [ ]:
train_set = CustomImageDataset("mnist_train.csv")
test_set = CustomImageDataset("mnist_test.csv")

batch_size = 64
shuffle = False

train_dataloader = DataLoader(train_set, batch_size=batch_size, shuffle=shuffle)
test_dataloader = DataLoader(test_set, batch_size=batch_size, shuffle=shuffle)

当数据集加载到 `DataLoader` 对象中后，您可以检查它们。对于本教程，数据集以64个数据集行的批次提供给训练应用程序。每个数据集行包含一个28x28的图像和一个标签（一个介于0和9之间的值）。

In [ ]:
for batch, (X, y) in enumerate(train_dataloader):
    print(len(X))
    print(len(y))

    first_image = X[0]
    first_label = y[0]

    print(len(first_image))
    print(first_label)  # This is a Tensor object with a single scalar value, 5
    break

从数据集中绘制一幅图像。

为了验证数据质量，绘制数据集中的第一项，以验证它是否呈现出与标签匹配的图像。

In [ ]:
first_image, label = (None, None)
for i in range(len(train_set)):
    sample = train_set[i]
    sample, label = sample
    first_image = sample.numpy()
    break

In [ ]:
np.shape(first_image)
imgplot = plt.imshow(first_image, cmap="gray")

## [可选] 在本地训练神经网络

虽然在 Vertex AI 上训练不需要这一步骤，但你可以使用 PyTorch 在本地训练模型。本教程声明了一个 `NeuralNetwork` 类，它继承自 PyTorch 的 [`torch.nn.Module`](https://pytorch.org/docs/stable/generated/torch.nn.Module.html?highlight=nn+module#torch.nn.Module) 类。`nn.Module` 类为所有神经网络模块提供了一个基类。

In [ ]:
# Get cpu or gpu device for training
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps" if torch.backends.mps.is_available() else "cpu"
)
print(f"Using {device} device")


# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28 * 28, 512, dtype=torch.float),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits


model = NeuralNetwork().to(device)
print(model)

In [ ]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        pred = model(X)
        loss = loss_fn(pred, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test(dataloader, model, loss_fn) -> bool:
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    accuracy = 100 * correct
    print(f"Test Error: \n Accuracy: {(accuracy):>0.1f}%, Avg loss: {test_loss:>8f} \n")
    if accuracy <= 0.0:
        return False
    return True


# Define a loss function and an optimizer.
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    is_going_well = test(test_dataloader, model, loss_fn)
    if not is_going_well:
        print("unacceptable accuracy")
        break
print("Done!")

## 创建训练脚本

不管其他的事情，训练自定义PyTorch模型在Vertex AI上的主要任务是创建一个训练脚本。这个脚本被加载到一个[用于PyTorch训练的预构建容器](https://cloud.google.com/vertex-ai/docs/training/pre-built-containers#pytorch)中，然后作为一个[在Vertex AI训练服务上运行的自定义训练作业](https://cloud.google.com/vertex-ai/docs/training/create-custom-job)运行。

第一步是为您的自定义训练作业选择一组兼容的加速器和训练图像。

In [ ]:
TRAIN_GPU, TRAIN_NGPU = (aiplatform.gapic.AcceleratorType.NVIDIA_TESLA_T4, 1)
if TRAIN_GPU:
    TRAIN_VERSION = "pytorch-gpu.1-13"
else:
    TRAIN_VERSION = "pytorch-xla.1-11"

TRAIN_IMAGE = "{}-docker.pkg.dev/vertex-ai/training/{}:latest".format(
    LOCATION.split("-")[0], TRAIN_VERSION
)

MACHINE_TYPE = "n1-standard"

VCPU = "4"
TRAIN_COMPUTE = MACHINE_TYPE + "-" + VCPU
print("Train machine type", TRAIN_COMPUTE)

### 编写训练脚本

接下来，在创建训练作业之前，您需要将训练脚本命名为'task.py'并保存到文件中。请注意，该脚本包含您之前检查过的数据集、数据加载器和神经网络模块。

在训练脚本中，训练脚本是使用[存储FUSE](https://cloud.google.com/storage/docs/gcs-fuse)从云存储加载的。FUSE将云存储桶挂载为训练容器文件系统中的文件夹。这使得训练脚本能够加载存储在存储桶中的文件作为数据集。FUSE还允许训练脚本将训练的输出，即模型产物，存储在云存储桶中。

要使用通过FUSE挂载到容器的存储桶，您需要用文件夹路径`/gcs/`替换存储桶URI中的`gs://`部分。

In [ ]:
# Make folder for Python training script
if not os.path.exists("trainer"):
    os.mkdir("trainer")

In [ ]:
%%writefile trainer/task.py
import os
import argparse
import logging

import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

parser = argparse.ArgumentParser(description='PyTorch CNN Training')
parser.add_argument('--train_uri', dest='train_uri',
                    type=str, help='Storage location of training CSV')
parser.add_argument('--test_uri', dest='test_uri',
                    type=str, help='Storage location of test CSV')
parser.add_argument('--model-dir', dest='model_dir',
                    default=os.getenv('AIP_MODEL_DIR'), type=str, help='Model directory')
parser.add_argument('--batch_size', dest='batch_size',
                    type=int, default=16, help='Batch size')
parser.add_argument('--epochs', dest='epochs',
                    type=int, default=20, help='Number of epochs')
parser.add_argument('--lr', dest='lr',
                    type=int, default=20, help='Learning rate')
args = parser.parse_args()

logging.getLogger().setLevel(logging.INFO)

class CustomImageDataset(Dataset):
    width = 28
    height = 28

    def __init__(self, data_file, transform=None, target_transform=None):
        self.dataset = pd.read_csv(data_file)
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return self.dataset.shape[0]

    def __getitem__(self, idx):
        label = self.dataset.at[idx, "label"]
        image = self.dataset.iloc[idx,1:]

        # Create a matrix from the pandas.Series
        image = image.to_numpy() * 0.00392156862745098 # 1 / 255
        image = image.reshape(self.width, self.height)
        image = image.astype(float)
        image = torch.Tensor(image)

        if self.target_transform:
            label = self.target_transform(label)
        if self.transform:
            image = self.transform(image)
        return image, label

# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512, dtype=torch.float),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

def get_data(train_gcs_uri, test_gcs_uri):

    train_set = CustomImageDataset(train_gcs_uri)
    test_set = CustomImageDataset(test_gcs_uri)

    # HARDCODED batch_size and shuffle-can customize
    batch_size = 64
    shuffle = False

    train_dataloader = DataLoader(train_set, batch_size=batch_size, shuffle=shuffle)
    test_dataloader = DataLoader(test_set, batch_size=batch_size, shuffle=shuffle)

    return train_dataloader, test_dataloader

def get_model():
    logging.info("Get model architecture")
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    gpu_id = "0" if torch.cuda.is_available() else None
    logging.info(f"Device: {device}")

    model = NeuralNetwork()
    model.to(device)

    loss = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)
    return model, loss, optimizer, device

def train_model(model, loss_func, optimizer, train_loader, test_loader, device):
    def train(dataloader, model, loss_fn, optimizer):
        size = len(dataloader.dataset)
        model.train()
        for batch, (X, y) in enumerate(dataloader):
            X, y = X.to(device), y.to(device)

            pred = model(X)
            loss = loss_fn(pred, y)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            if batch % 100 == 0:
                loss, current = loss.item(), batch * len(X)
                print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

    def test(dataloader, model, loss_fn):
        size = len(dataloader.dataset)
        num_batches = len(dataloader)
        model.eval()
        test_loss, correct = 0, 0
        with torch.no_grad():
            for X, y in dataloader:
                X, y = X.to(device), y.to(device)
                pred = model(X)
                test_loss += loss_fn(pred, y).item()
                correct += (pred.argmax(1) == y).type(torch.float).sum().item()
        test_loss /= num_batches
        correct /= size
        accuracy = 100 * correct
        print(f"Test Error: \n Accuracy: {(accuracy):>0.1f}%, Avg loss: {test_loss:>8f} \n")

    # Define a loss function and an optimizer.
    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

    epochs = 5
    for t in range(epochs):
        print(f"Epoch {t+1}\n-------------------------------")
        train(train_loader, model, loss_fn, optimizer)
        test(test_loader, model, loss_fn)

    # Done training
    return model

# import data from Cloud Storage
logging.info('importing training data')
gs_prefix = 'gs://'
gcsfuse_prefix = '/gcs/'

if args.train_uri.startswith(gs_prefix):
    args.train_uri.replace(gs_prefix, gcsfuse_prefix)

if args.test_uri.startswith(gs_prefix):
    args.test_uri.replace(gs_prefix, gcsfuse_prefix)

train_dataset, test_dataset = get_data(train_gcs_uri=args.train_uri,
                                      test_gcs_uri=args.test_uri)

logging.info('starting training')
model, loss, optimizer, device = get_model()
train_model(model, loss, optimizer, train_dataset, test_dataset, device)


# export model to gcs using GCSFuse
logging.info('start saving')
logging.info("Exporting model artifacts ...")
gs_prefix = 'gs://'
gcsfuse_prefix = '/gcs/'
if args.model_dir.startswith(gs_prefix):
    args.model_dir = args.model_dir.replace(gs_prefix, gcsfuse_prefix)
    dirpath = os.path.split(args.model_dir)[0]
    if not os.path.isdir(dirpath):
        os.makedirs(dirpath)

gcs_model_path = os.path.join(os.path.join(args.model_dir, 'model.pth'))
torch.save(model.state_dict(), gcs_model_path)
logging.info(f'Model is saved to {args.model_dir}')

### 创建训练任务

一旦您将训练脚本写入文件，现在可以开始训练模型了。对于这个模型，在调用 `CustomTrainingJob.run()` 时提供以下参数。还请注意，`args` 列表中提供的字符串是训练脚本中定义的参数。

+ `--train_uri` 和 `--test_uri` 参数指向一个公开可用的 Cloud Storage 存储桶中的 CSV 文件。训练脚本使用 Storage FUSE 访问这些文件。
+ `--model_dir` 参数指向一个您必须提供给脚本的存储桶。训练脚本会在存储桶上创建一个新文件夹来存储模型构件。

In [ ]:
# Use timestamped path to save your model in Cloud Storage
TIMESTAMP = datetime.now().strftime("%Y%m%d-%H%M%S")
# Set a display name for the training job
JOB_DISPLAY_NAME = "pytorch-custom-job"

# Create a custom training job in Vertex AI
job = aiplatform.CustomTrainingJob(
    display_name=JOB_DISPLAY_NAME,
    script_path="trainer/task.py",
    container_uri=TRAIN_IMAGE,
)

# Run the job
job.run(
    replica_count=1,
    machine_type=TRAIN_COMPUTE,
    accelerator_type=TRAIN_GPU.name,
    accelerator_count=TRAIN_NGPU,
    args=[
        "--train_uri",
        TRAIN_URI,
        "--test_uri",
        TEST_URI,
        "--model-dir",
        f"{BUCKET_URI}/{BUCKET_PREFIX}/{TIMESTAMP}/",
    ],
)

清理

要清理此项目中使用的所有Google Cloud资源，您可以[删除用于教程的Google Cloud项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。否则，您可以删除在本教程中创建的各个资源。

In [ ]:
# Delete training job created
job.delete(sync=False)

# Delete Cloud Storage objects that were created
delete_bucket = False
if delete_bucket:
    ! gsutil -m rm -r $BUCKET_URI